In [1]:
import pandas as pd
import re

from maple.types import Action, ActionType
from maple.classification import parse

import warnings
warnings.filterwarnings('ignore')


In [2]:
# legislators

# list is incomplete -- used only for research purposes
legislators = [
"Gomez",
"Cronin",
"Gobi",
"Crighton",
"Tarr",
"Stone", 
"Creem",
"Lesser",
"Chandler",
"Boncore",
"Eldridge",
"Lewis",
"Lovely",
"Comerford",
"Keenan",
"Edwards",
"Spilka",
"Brady",
"Montigny",
"Moore",
"Pacheco",
"Rodrigues",
"Rush",
"O'Connor",
"Jehlen",
"Rausch",
"Fattman",
"Chang-Diaz",
"DiDomenico",
"Moran",
"Brownsberger",
"Timilty",
"Consalvo",
"Duffy",
"Giannino",
"Kushmerek",
"Pease",
"Philips",
"Scanlon",
"Xiarhos",
"Oliveira",
"Cabral",
"D'Emilia",
"Madaro",
"Ferrante",
"Michlewitz",
"Peisch",
"Puppolo",
"Silvia",
"Ashe",
"Ayers",
"Jones",
"Murray",
"González",
"Doherty",
"Barber",
"Cronin",
"Fiola",
"LeBoeuf",
"Cahill",
"Donahue",
"Garlick",
"Gregoire",
"Linsky",
"Tucker",
"Balser",
"Holmes",
"Mariano",
"Dooley",
"Garballey",
"Harrington",
"Howitt",
"Peake",
"Pignatelli",
"Ultrino",
"Whipps",
"Farley-Bouvier",
"Golden",
"Nguyen",
"Stanley",
"Walsh",
"Whelan",
"Galvin",
"Straus",
"Turco",
"Fernandes",
"Cassidy",
"Kane",
"Hawkins",
"Arciero",
"Wagner",
"Kelcourse",
"McGonagle",
"McKenna",
"O'Day",
"Parisella",
"Roy",
"Zlotnik",
"Lipper-Garabedian",
"Ferguson",
"Gordon",
"Hogan",
"Campbell",
"Ehrlich",
"Sabadosa",
"Malia",
"Mirra",
"Devers",
"Robinson",
"Cusack",
"Day",
"Decker",
"DuBois",
"Finn",
"Keefe",
"Muratore",
"Blais",
"Higgins",
"Boldyga",
"Donato",
"Durant",
"Haddad",
"Mark",
"Schmid",
"Frost",
"Belsito",
"Hinds",
"Finegold",
"Friedman",
"DiZoglio",
"Kennedy",
"Velis",
"Cyr",
"Barrett",
"Collins",
"Feeney",
"Diggs",
"Oakley",
"Ramos",
"Sullivan",
"Vargas",
"Williams",
"Hendricks",
"Minicucci",
"Sena",
"Berthiaume",
"Robertson",
"Rogers",
"Mom",
"Gouveia",
"Moran",
"Lewis"
]


hash_reg = re.compile('\(#.*\)')

def replace_legislator_names(action: str):
  matches = [l for l in legislators if l in action]
  
  for m in matches: 
    r = '\(.*' + m + '.*\)'
    action = "(###)".join(re.compile(r).split(action))
  action = "(leg_names)".join(hash_reg.split(action))
  return action


In [3]:
# committees

# TO DO check and complete these lists - update in january 
first_committees  = [
  "advanced information technology, the internet and cybersecurity",
  "bonding, capital expenditures and state assets", "cannabis policy" ,
  "children, families and persons with disabilities",
  "community development and small businesses",
  "consumer protection and professional licensure",
  "covid-19 and emergency preparedness and management",
  "economic development and emerging technologies",
  "higher education",
  "education",
  "elder affairs",
  "election laws",
  "environment, natural resources and agriculture",
  "export development",
  "financial services",
  "Housing and Food Security", 
  "Housing and Local Aid Social Services and Veteran Services and Soldiers",
  "Housing Energy and Environmental Affairs and com_name"
  "housing",
  "the judiciary",
  "labor and workforce development",
  "mental health substance use and recovery",
  "municipalities and regional government",
  "public health",
  "public safety and homeland security",
  "public service",
  "racial equity, civil rights and inclusion ",
  "revenue",
  "state administration and regulatory oversight",
  "telecommunications, utilities and energy",
  "tourism, arts and cultural development",
  "transportation",
  "Veterans and Federal Affairs",
  "House Bills in Third",
  "Election Laws",
  "Transportation",
  ]


chamber_committees = [
  "committee on rules",
  'committee on rules of the two branches, acting concurrently',
  "ways and means", 
  "steering, policy and scheduling",
  "steering and policy",
  "housing and finance",
  "health care financing",
  "Bills in the third reading"
]


def extract_committees(action):
  found_committees = []
  for f in first_committees:
    found_committees.extend(re.compile('(' + f + ')', re.I).findall(action))
  for c in chamber_committees:
    found_committees.extend(re.compile('(' + c + ')', re.I).findall(action))
  if len(found_committees) > 0: 
    return found_committees 


def extract_first_committees(action):
  found_committees = []
  for f in first_committees:
      m = re.compile('((Joint|House|Senate|Temporary )?committee on ' + f + ')', re.I).search(action)
      if m != None: 
        found_committees.append(m.group())
  if len(found_committees) > 0: 
    return found_committees
 
def extract_chamber_committees(action):
  found_committees = []
  for c in chamber_committees:
    m = re.compile('((Joint|House|Senate|Temporary )?committee on ' + c + ')', re.I).search(action)
    if m != None: 
      found_committees.append(m.group())
  if len(found_committees) > 0: 
    return tuple(found_committees)

def replace_committees(action):
  for f in first_committees:
    re_f = re.compile(f, re.I)
    if re_f.search(action) != None: 
      action ="[first_committee]".join(re_f.split(action))
  for c in chamber_committees:
    re_c = re.compile(c, re.I)
    if re_c.search(action) != None: 
      action ="[chamber_committee]".join(re_c.split(action))
  return action


In [4]:
# date words

day = ['sunday', 'monday', 'tuesday','wednesday', 'thursday', 'friday', 'saturday']
month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

def date_dates(action):
  for d in day: 
    dreg = re.compile(d, re.I)
    day_split = dreg.split(action)
    action = "[day]".join(day_split)
    
  for m in month: 
    mon_reg = re.compile(m, re.I)
    mon_split = mon_reg.split(action)
    action = "[month]".join(mon_split)
  
  date_reg = re.compile('\d\d/\d\d/\d\d\d\d', re.I)
  action = date_reg.sub("[date]", action)
  
  year_reg = re.compile('202\d', re.I)
  action = year_reg.sub("[year]", action)
  
  time_reg = re.compile('\d\d:\d\d(?:\s?[AP]M)?', re.I)
  action = re.sub(time_reg, "[time]", action)

  
  return action

In [5]:
# replacement functions

def bill_number_replace(action):
  reg = re.compile('(?:[HS]\d+)', re.I)
  a = reg.findall(action)
  if len(a) > 0: 
    breg = re.compile("(.*) ".join(a), re.I)
    action = breg.sub('[billnum]', action)
  return action

def number_replace(action):
  reg = re.compile('(?:\d+,? ?)+(?:and )?', re.I)
  action = re.sub(reg, '[num]', action)
  return action

def reletter_attachment(action): 
  reg = re.compile('attachment\s\w$', re.I)
  return re.sub(reg, 'Attachment X', action)

def reletter_amendment(action): 
  reg = re.compile('amendment\s\w\W', re.I)
  return re.sub(reg, 'amendment X ', action)

def renumber_amendment(action):
  find=re.compile('Amendment', re.I)
  replace = re.compile('#?\d+\.?\d*')
  if find.search(action):
    action = "[num]".join(replace.split(action))
  return action

def comma_spacer(action):
  reg = re.compile(',\s?', re.I)
  return re.sub(reg, ', ', action)

def consolidated_amendment(action):
  consolidated_amendment = re.compile('consolidated amendment', re.I)
  replace_inside_parens = re.compile('\([^S].*\)')
  if consolidated_amendment.match(action) != None:
    action = "([com_name])".join(replace_inside_parens.split(action))
  return action  
  
def replace_vote(action):
  find = re.compile('YEAS.*NAYS', re.I)
  reg = re.compile('[\s-]*\d*\s?\(?YEAS to \d+\s?NAYS\)?\s?\(.*\)', re.I)
  if find.search(action) != None: 
      splits = reg.split(action)
      action = " [vote]".join(splits)
  return action


def norm_spellings(action):
  words = [
    ('therefor.', 'therefor'), 
    ("in the place", "in place"), 
    ("  ", " "), 
    ('committee on conference', 'committee of conference'), 
    ('\s?-\s?', ' ') ]
  for a in words: 
    action = re.sub(a[0], a[1], action, re.I )
  return action 


def amendments_considered(action):
  reg = re.compile('^(The following amendments were considered as one and \w*)', re.I)
  capture = reg.search(action)
  if capture != None: 
      action = capture.group() + ' [amdmnts]'
  return action




In [6]:
# apply replacement functions

df = pd.read_csv("predictions.csv")

# df = pd.read_csv("../../legislative-testimony/advocacy-maps/analysis/data/all-history-actions.csv")

df['clean_actions'] = df['action']
df['clean_actions'] = df['clean_actions'].apply(lambda s: bill_number_replace(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: replace_legislator_names(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: comma_spacer(s))
df['first_committees'] = df['clean_actions'].apply(lambda row: extract_first_committees(row)) 
df['chamber_committees'] = df['clean_actions'].apply(lambda row: extract_chamber_committees(row)) 
df['clean_actions'] = df['clean_actions'].apply(lambda row: replace_committees(row)) 
df['clean_actions'] = df['clean_actions'].apply(lambda s: date_dates(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: reletter_attachment(s))

df['clean_actions'] = df['clean_actions'].apply(lambda s: replace_vote(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: reletter_amendment(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: renumber_amendment(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: amendments_considered(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: consolidated_amendment(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: norm_spellings(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: number_replace(s))



In [7]:
# format output and remove duplicates

pd.options.display.max_colwidth = -1


drp = df.drop_duplicates(subset=['clean_actions']) #.sort_values(by='clean_actions')



In [8]:
labels= [
  # identifying committees
  'ho.se', 
  'senate',
  'committee(?: o[nf] conference)?',
  'first_committee',
  'chamber_committee',
  'com_name',
  
  # while in committee
  'hearing',
  'schedule',
  'postpone[d]?',
  'reschedule',
  'cancel',
  'calendar',
  
  # committee reports/outcomes
  'study order', 
  'discharged', 
  'placed on file',
  'accompan\w*(?:\sby)?',
  'refer\w*(?: to)?(?: from)?',
  'new draft',
  'new text', 
  'recommend\w*',  
  'substitut\w*',  
  'substituted for',
  'see',
  'see [billnum]',

  
  'bill reported favorably', 
  'bill reported adversely', 
  'reported',
  'ought to pass',
  'ought not to pass', 
  'ought to be adopted', 
  'ought not to be adopted', 
  'ought to be approved', 
  'ought not to be approved', 
  
  
  
  'recommitted',
  
  'report accepted', 
  'reported to', 
  'reported from'
  
  'appointed',


# amendment stuff  
  'striking out(?: all)?',
  'pending',
  '(?:published as )?(?:printed as )?amend\w*(?: accepted)?(?: considered)?', 
  'adopt\w*', 
  'concur\w*',
  'printed as amended',
  'reprint\w*(?: as amended)?'
  
# chamber committees
  'house ways and means',
  'bills in the third reading',
  'steering, policy and scheduling',
  'steering and policy',
  'health care finance',
  'Bonding, Capital Expenditures and Assets',
  'committee on rules of the two branches, acting concurrently',
  'committee on rules',

# chamber sessions
  'order\w*(?: of the day)?(?: to a third reading)?',
  'leg_names',
  'rejected',
  'laid aside',
  'enacted', 
  'engrossed',
  'joint session',
  '^read$',
  'read second',
  'read third',
  'first reading',
  'second reading',
  'suspended',
  'rules suspended',
  'point of order',
  'negatived',
  'returned',
  
  
  
# votes
  'vote', 
  'Roll Call',
  'passed over veto',
  
  
  
# governor
  'laid before the governor',
  'signed by the governor',
  'attachment',
  'governor',
  'signed',
  'in part',
  'chapter',
  'message',
  'motion', 
  'consolidated',
  
  
  
  # pick ups
  'Transmitted to the Secretary of State',
  'General Appropriation Bill',
  'emergency preamble',
  'constitutional amendment',
  'Taken out of the notice section',
  'also based on',
  'read for its information',
  'duplicate'
  
  ]



def set_labels(action):
  clean_labels: list(str) = []
  for l in labels:
    reg = re.compile(l, re.I)
    m = reg.search(action)
    if m != None:
      lab = m.group().lower()
      clean_labels = clean_labels + [lab]
  return clean_labels
  


drp['label'] = drp['clean_actions'].apply(lambda s: set_labels(s))


In [27]:
action_types = [
  (['new draft'], 'bill redirect'),
  (['new text'], 'bill redirect'),
  (['study order'], 'bill redirect'), 
  (['accompanied'], 'bill redirect'),
  (['accompanied by'], 'bill redirect'),
  (['reported, in part'], 'bill redirect'),
  (['reported in part'], 'bill redirect'),
  (['reported', 'in part'], 'bill redirect'),
  (['accompanying'], 'bill redirect'),
  (['hearing scheduled'], 'needs context'),
  (['hearing rescheduled'], 'needs context'),
  (['bill reported favorably'],'committee report'),
  (['committee reported favorably'],'committee report'),
  (['bill reported unfavorably'],'committee report'),
  (['committee reported unfavorably'],'committee report'),
  (['committee reported'], 'committee report'),
  (['reported'], 'committee report'),
  (['ought to pass'], 'committee report'),
  (['recommending'], 'committee report'),
  (['ought not to pass'], 'committee report'),
  (['ought to be adopted'], 'committee report'),
  (['committee recommended'], 'committee report'),
  (['recommitted'], 'referred to committee'),
  (['referred to the committee'], 'referred to committee'), 
  # NOTE could be either joint committees or chamber committees 
  
  (['read second'], 'chamber business'),
  (['read third'], 'chamber business'),
  (['second reading'], 'chamber business'),
  (['third reading'], 'chamber business'),
  (['ordered to a second reading'], 'chamber business'),
  (['ordered to a third reading'], 'chamber business'),
  (['orders of the day'], 'chamber business'),
  (['bills in the third reading'], 'chamber business'),
  (['read'], 'read'),
  
  
  (['amendment'], 'amendment'),
  (['amendments'], 'amendment'),
  (['amended'], 'amended'),
  (['laid aside'], 'laid aside'),
  (['rejected'], 'rejected'),
  (['accepted'], 'accepted'),
  (['adopted'],  'adopted'),
  (['striking out'], 'striking out'),
  (['striking out all'], 'striking out all'),
  (['printed as amended'],'printed as amended'),
  (['reprinted as amended'],'reprinted as amended'),
  
  (['engrossed'], 'engrossment'),
  (['enacted'], 'enactment'),
  (['laid before the governor'], 'laid before the governor'),
  (['message'], 'message'),


  (['first_committee'], 'first committee'),
  (['chamber_committee'], 'chamber committee'),
  
  (['committee', 'referred'], 'referred to committee'),
  (['committee', 'referred to'], 'referred to committee'),
  (['committee', 'referred', 'bills in the third reading'], 'to bills in the third reading'),
  (['committee', 'referred to', 'bills in the third reading'], 'to bills in the third reading'),
  (['committee', 'ought to pass'], 'committee report'),
  (['committee', 'ought not to pass'], 'committee report'),
  (['committee', 'reported favorably'], 'committee report'),
  (['committee', 'reported unfavorably'], 'committee report'),
  (['calendar'], 'scheduling'),
  (['calendar','joint session'], "scheduling"),
  (['accompanied', 'new draft', 'see'], 'rewritten'),
  (['amendment', 'rejected'], 'amendment rejected'),
  (['amendment', 'adopted'], 'amendment adopted'),
  (['amendment', 'rejected', 'vote'], 'amendment rejected by vote'),
  (['amendment', 'adopted', 'vote'], 'amendment adopted by vote'),
  (['report', 'accepted'], 'report accepted'),
  (['governor', 'recommend\w*'], 'governor recommendation'),
  (['signed by the governor'], 'signed by the governor'),
  
  (['taken out of the notice section'], 'no status'),
  (['also based on'], 'no status'),
  (['read for its information'], 'no status'),
  (['read'], 'read'),
  (['duplicate'], 'duplicate')
]



def get_action_types(label):
  acts = []
  for acttype in action_types: 
    if all([a.lower() in label for a in acttype[0]]):
      acts.append(acttype[1])
  if len(acts) > 0: 
    return acts
  return ['no status']
      


drp['action_type'] = drp['label'].apply(lambda l: get_action_types(l))


In [28]:
# prep output

output = drp[[ 'action_id', 'action', 'prediction', 'action_type']]

output['action_type'] = output['action_type'].apply(lambda x: pd.Series(x).unique())

def outputSingleType(a_type):
    type_name = a_type.replace(" ", "_")
    print(type_name, a_type)
    df_type_filtered = output[output['action_type'].apply(lambda x: tuple(x).count(a_type) > 0)]
    df_type_filtered['action_type'] = df_type_filtered['action_type'].apply(lambda x: type_name)
    df_type_filtered.set_index('action_id', inplace=True)
    file_path = f'./actiontype_assgns/{type_name}.csv'
    df_type_filtered.to_csv(file_path)

In [32]:
# output

outputSingleType('first committee')
outputSingleType('chamber committee')
outputSingleType('bill redirect')
# bill number will very often change or die at a redirect

outputSingleType('laid before the governor')
outputSingleType('chamber business')
outputSingleType('committee report')
outputSingleType('engrossment')
# engrossement to cross over depends on being in the 1st chamber
# 2+ engrossment doesn't change it's tracker status




first_committee first committee
chamber_committee chamber committee
bill_redirect bill redirect
laid_before_the_governor laid before the governor
chamber_business chamber business
committee_report committee report
engrossment engrossment
